# Défenses adversariales

Cette fois-ci, nous allons nous consacrer à la défense des réseaux aux attaques adversariales. 
Pour cela nous implémentons une méthode qui permet de résister à ces exemple tout en étant assez simple à implémenter: l'entraînnement adversarial.

## Bibliographie

[1](https://arxiv.org/pdf/1611.01236.pdf) Kurakin, A., Brain, G., Openai, I. J. G., & Bengio, S. (n.d.). ADVERSARIAL MACHINE LEARNING AT SCALE.

## Librairies Python

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as dataset
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
import time
import datetime

## Données et modèles

Comme au TP précédent, nous définissons notre modèle et nous chargeons les données d'entraînement.

In [3]:
# MNIST Test dataset and dataloader declaration
mnist_train = dataset.MNIST('./data', train=True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),]))
mnist_test = dataset.MNIST('./data', train=False, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),]))

In [4]:
# LeNet Model definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

Au dernier TP, nous n'avons pas eu à faire d'entraînement et nous n'avons donc pas défini des paramètres liés à cela comme l'optimiseur, le taux d'apprentissage, le nombre d'epochs, etc. Cette fois-ci, nous allons effectuer un entraînement, nous devons donc définir certains paramètres pour lesquels nous pouvons prendre des valeurs assez classiques :
<ul>
    <li>Taille de batchs : 32 (tombe bien car nous avons 32x1875 images d'entraînement)</li>
    <li>Fonction de perte : negative log likelihood loss (utilisée en classification)</li>
    <li>Nombre d'epochs (nombre de fois que la totalité des données d'entraînement seront "passées" par le réseau) </li>
    <li>Taux d'apprentissage : 0.01</li>
    <li>Optimiseur : SGD (descente de gradient stochastique)</li>
</ul>

Il faudra ensuite créer les fonctions `train_model` et `test_model` qui nous permettront d'entraîner et d'évaluer des modèles respectivement.

In [5]:
train_batch_size = 
loss_function = 
epochs = 
lr = 

In [6]:
# Données par batch
trainloader = torch.utils.data.DataLoader(mnist_train, batch_size=train_batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(mnist_test, batch_size=1, shuffle=True)

## Entraînement du modèle

In [7]:
def train_model(model, optimizer, dataloader, epochs, lossF):
    """
    model : réseau à entraîner
    optimizer : optimiseur du réseau
    dataloader : données d'entraînement (images, labels) par batchs
    epochs : nombre d'epochs d'entraînement
    lossF : fonction de perte
    """
    s = time.time()
    
    for epoch in range(epochs):
        
        for inputs, labels in dataloader:

            # Mettre à zero les gradients de l'optimiseur pour éviter qu'ils s'accumulent
            optimizer.zero_grad()
            
            # Passer les données d'entrée au modèle
            output = model(inputs)
            
            # Calculer la fonction de perte pour ces données et les gradients correspondants 
            loss = loss_fonction(output, labels)
            loss.backward()
            
            # Effectuer un "pas" de l'optimisation
            optimizer.step()
            
    print('Finished training, took {}'.format(datetime.timedelta(seconds=time.time() - s)))
    
    return model

In [8]:
def test_model(model, dataloader):
    """
    model : réseau à évaluer
    dataloader : données de test (images, labels) par batchs
    """
    with torch.no_grad():
        model.eval()
        # Créer un compteur pour stocker les prédictions correctes

    
        # Parcourir les données d'évaluation
        for inputs, target in dataloader:

            # Passer les données d'entrée au modèle et récupérer la prédiction


            # Si la prédiction est correcte, la comptabiliser 

        
    print('Accuracy of model : {}'.format(correct/len(dataloader)))    

Le but de ce TP est de comparer la robustesse aux attaques d'un réseau entraînné sur des exemples non bruités et d'un réseau entraînné avec des exemples adversariaux. Commençons par l'entraînnement "classique" d'un réseau :

In [25]:
# Définir un réseau


# Créer un optimiseur pour les poids du réseau


# Entraîner et évaluer le modèle



Finished training
Accuracy of model : 0.8774


## Entraînement adversarial

Maintenant nous pouvons entraîner un autre réseau avec des exemples adversariaux. 

Le pseudo code de l'entraînnement adversarial ([1](https://arxiv.org/pdf/1611.01236.pdf)) est le suivant :
1. Initialiser aléatoirement un réseau (fait par Pytorch)
2. Pour chaque minibatch B = {$X^1$,..., $X^m$} du set d'entraînement :
    * Générer $k$ exemples adversariaux {$X^1_{adv}$,..., $X^k_{adv}$} à partir des exemples non bruités originaux {$X^1$,..., $X^k$}
    * Former un nouveau minibatch B' = {$X^1_{adv}$,..., $X^k_{adv}$, $X^{k+1}$,..., $X^m$}
    * Effectuer l'entraînement du réseau en utilisant le nouveau minibatch B'
3. Répéter jusqu'à convergence

Ici, nous allons attaquer le réseau avec l'algorithme FGSM vu précédement (mais il peut s'agir de n'importe quelle autre attaque). Vous pouvez créer les fonctions `fgsm_attack` et `adversarial_training` qui devront respectivement retourner une image bruitée et effectuer un entraînement adversarial.

In [9]:
def fgsm_attack(model, image, target, lossF, epsilon):
    """
    model : réseau à attaquer
    image : image qui sera bruitée
    target : label de l'image
    lossF : fonction de perte 
    epsilon : paramètre de l'attaque
    
    Return:
        noised_image : image bruitée
    """
    # L'attribut requires_grad du tensor des données doit être vrai pour pouvoir calculer
    # le gradient de la fonction de perte par rapport à celui-ci

        
    # Récupérer la prédiction du modèle

    
    # Calculer la fonction de perte et ses gradients

    
    
    # Récupérer le signe des coefficients du gradient par rapport aux données d'entrée

    
    # Créer l'image bruitée à partir de l'image en entrée

    
    # Borner l'image obtenue entre 0 et 1 (utiliser torch.clamp)

    
    # Retourner l'image bruitée


In [10]:
def adversarial_training(model, optimizer, dataloader, epochs, lossF, k, attack, epsilon):
    """
    model : réseau à attaquer
    optimizer : optimiseur associé au réseau
    dataloader : données d'entraînement (images, labels) par batchs*
    epochs : nombre d'epochs d'entraînement
    lossF : fonction de perte 
    k : nombre d'exemples adversariaux à générer
    attack : fonction de l'attaque à appliquer
    epsilon : paramètre de l'attaque
    """
    s = time.time()
    for epoch in range(epochs):
        
        for inputs, labels in dataloader:
            
            # Selectionner les k premières images à bruiter et leurs labels

            
            
            # Bruiter ces images avec l'attaque choisie
            
            
            # Concaténer ces k images bruitées aux (N-k) images non-bruitées
            
            
            # Mettre à zero les gradients de l'optimiseur pour éviter qu'ils s'accumulent
            
            
            # Passer les données d'entrée au modèle
            
            
            # Calculer la fonction de perte pour ces données et les gradients correspondants 
            
            
            # Effectuer un "pas" de l'optimisation  
            
    print('Finished adversarial training, took {}'.format(datetime.timedelta(seconds=time.time() - s)))

Vous pouvez maintenant créer un deuxième réseau, l'entraînner avec des exemples adversariaux et évaluer ses performaces.

In [29]:
# Définir un réseau et les paramètres de l'attaque
net_adv = 
k = 
epsilon = 


# Créer un optimiseur pour les poids du réseau


# Entraîner le modèle avec des exemples adversariaux et l'évaluer



Finished training
Accuracy of model : 0.8677


## Attaque des réseaux

Nous avons à présent deux réseaux qui atteignent des performances correctes. Nous allons créer une fonction `test_adversarial` qui permettra d'attaquer des réseaux et qui affichera leur performance après l'attaque.

In [11]:
def test_adversarial(model, dataloader, lossF, attack, epsilon):
    """
    model : réseau à attaquer
    dataloader : données de test (images, labels) par batchs
    lossF : fonction de perte 
    attack : fonction de l'attaque à appliquer
    epsilon : paramètre de l'attaque
    """

    # Variable qui va nous permettre de stocker les exemple robustes
    robust_examples = 

    # Parcourir les examples dans les données d'entraînnement
    for inputs, target in dataloader:

        # Récupérer la prédiction du modèle
        output = 
        pred = 
        
        # Attaquer uniquement des classifications correctes
        if pred.item() == target.item():
            
            # Bruiter l'image avec l'attaque choisie
            noised_image = 
            
            # Re-classifier l'image bruitée et récupérer sa prédiction
            new_output = 
            attacked_pred = 
            
            # Stocker les exemples robustes
            if 
            
   
    # Calculer la précision finale après l'attaque (les exemples robustes)
    final_acc = robust_examples/float(len(dataloader))
    print("Epsilon: {}\tTest Accuracy with attack = {} / {} = {}".format(epsilon, robust_examples, len(dataloader), final_acc))

Nous pouvons maintenant comparer les deux réseaux et voir lequel est plus robuste aux attaques. Vous pouvez jouer sur des paramètres comme $\epsilon$, le nombre d'epochs, le nombre d'exemples adversariaux $k$,... (modifier certains paramètres nécésite un re-entraînement...)

In [31]:
# Attaque du réseau "normal"
test_adversarial(net, testloader, loss_function, fgsm_attack_step, epsilon)

Epsilon: 0.3	Test Accuracy with attack = 1331 / 10000 = 0.1331


In [32]:
# Attaque du réseau entraîné avec des exemples adversariaux
test_adversarial(net_adv, testloader, loss_function, fgsm_attack_step, epsilon)

Epsilon: 0.3	Test Accuracy with attack = 3583 / 10000 = 0.3583
